### Vertical Regridding 


**The math**

An example of vertical regridding is to take a model that has depth (z) as its vertical grid, and translate it to some tracer (c (x,y,z,t)): $f(x,y,z,t) \rightarrow \tilde{f}(x,y,\tilde{c},t)$. This operation can be written cleanly using dirac-delta functions:

\begin{equation}
\tilde{f}(x,y, \tilde{c},t) = \frac{1}{\sigma} \int f(x,y, z,t) \delta [c(x,y,z,t) - \tilde{c}] dz
\end{equation}

where $\sigma$ is the thickness defined as:
\begin{equation}
\sigma  = \int \delta (c(x,z,t) - \tilde{c}) dz
\end{equation}

and the integrals are evaluated over the entire depth. When an integrand is multiplied by the dirac-delta function, the integration only picks up the values of the integrand at places where the dirac-delta function's argument is zero (or the location where tracer value (c) is the same as the tracer value that is being projected on ($\tilde{c}$)).

The division by the thickness can be understood by an **example** and thinking discretely. Imagine that in a column of water that is 1000m deep there is a tracer that has value 1 between 600 and 700m, and zero everywhere else. Let's say the function that needs to be regridded is the velocity. For $\tilde{c} =1$, the integral will give us a vertical sum of the velocity b/w 600 and 700m ($\int_{600}^{700} u dz$), which will have units of $[L^2]/T$. To find the average velocity on the region where the tracer has value 1, we need to divide by the thickness (units $[L]$) of this vertical region ($\sigma(\tilde{c}) = \int_{600}^{700} dz = 100m$). (An an exercise you can think of what happens for $\tilde{c} =0$.)

**Numerical Implementation**

Since, in practice it is almost impossible to work with dirac-delta functions, we instead rely on bins. So $\hat{f} (x,y, \hat{c},t)$ is an approximation to $\tilde{f}(x,y, \tilde{c},t)$, and $\hat{c}$ is an approximation to $\tilde{c}$ that allows values in some binned range $\hat{c} = \tilde{c} \pm \triangle \tilde{c}$.

One way to do this binning is by making histograms. A histogram function finds all points with values that lie in a certain range of bin, which is like gathering the locations on the grid that will be used in the integral (a discrete approximation to the dirac-delta function). These values can then be weighted by $dz$ and the corresponding function values $f$ at the points found by histogram to estimate the integral. 

This is the logic in Julius's function, reproduced below:

In [2]:
def vertical_sum(data, bin_data, bins, dz, vert_dim="st_ocean"):
    """ This function calculates the vertical sum of a variable but weighted by the dirac-delta
    functions, as shown in the first equation. """
    
    nanmask = np.isnan(data)
    # Should we also check the bin data for nans?
    full_sum = histogram(
        bin_data.where(~nanmask),
        bins=[bins],
        weights=(data * dz).where(~nanmask),
        dim=[vert_dim],
    )
    return full_sum

def vertical_rebin_wrapper(
    ds,
    bin_data_name,
    bins,
    dz_name="dz",
    vert_dim="st_ocean",
    return_average=True,
    debug=False,
):
    """A wrapper for the core functionality in `vertical_rebin`.
    Accepts datasets and calculates the average over the new depth coordinates.
    
    Inputs:
    ds - the Dataset with one of the variables being the tracer that would be the new grid.
    tracer_name - the tracer variable name to which the regridding will be done. 
    bins - the discrete tracer bins into which will form the ranges for the new vertical grid.
    dz_name  - the variable name for the cell sizes on a z grid (or cell sizes for current vertical grid)
    vert_dim - current vertical dimension of the data
    
    Outputs: 
    A new data set where all variables have been regridded to a new vertical grid.
    """
    
    
    ds = ds.copy() # the input
    ds_rebinned = xr.Dataset() # the output
    
    ones = xr.ones_like(ds[dz_name]) 
    
    # Estimate the thickness of each tracer bin.
    # this is the calculation of sigma
    thickness = vertical_sum(
        ones,
        ds[bin_data_name],
        bins,
        ds[dz_name],
        vert_dim=vert_dim,
    )
    
    # Estimate the vertical integrals in the numerator
    for var in ds.data_vars:
        ds_rebinned[var] = vertical_sum(
            ds[var], ds[bin_data_name], bins, ds[dz_name], vert_dim=vert_dim
        )
        
    if return_average:
        ds_rebinned = (
            ds_rebinned / dz_rebinned
        )  
        # this might cause a lot of overhead...i can try to deactivate if the save fails.
        # this is needed, or else ds_rebinned has the wrong units (multiplied by thickness).

    ds_rebinned['thickness'] = thickness

    return ds_rebinned